In [5]:
!pip install requests tqdm

     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ---------------------------------------- 78.5/78.5 kB 4.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
API_KEY=""

In [6]:
import os
import time
import json
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_columns', None)

In [ ]:
#get match data given match_id
def get_match(matchId, region_long):
  link = f'https://{region_long}.api.riotgames.com/tft/match/v1/matches/{matchId}?api_key={API_KEY}'
  response = requests.get(link)
  if(response.status_code == 200):
    return response.json()
  else:
    return response.status_code

In [ ]:
CONTENT_PATH = '..\data\MATCHIDS_PATCH13.4_NA_BR_EUW_KR.json' #read in match ids
with open(CONTENT_PATH, 'r') as file:
    matchIds = json.load(file)

In [ ]:
columns = [ "Champion 1", "Level 1", "Item 1 1", "Item 1 2", "Item 1 3", "Champion 2", "Level 2", "Item 2 1", "Item 2 2", "Item 2 3",
           "Champion 3", "Level 3", "Item 3 1", "Item 3 2", "Item 3 3", "Champion 4", "Level 4", "Item 4 1", "Item 4 2", "Item 4 3",
           "Champion 5", "Level 5", "Item 5 1", "Item 5 2", "Item 5 3", "Champion 6", "Level 6", "Item 6 1", "Item 6 2", "Item 6 3",
           "Champion 7", "Level 7", "Item 7 1", "Item 7 2", "Item 7 3", "Champion 8", "Level 8", "Item 8 1", "Item 8 2", "Item 8 3",
           "Champion 9", "Level 9", "Item 9 1", "Item 9 2", "Item 9 3", "Champion 10", "Level 10", "Item 10 1", "Item 10 2", "Item 10 3",
           "Champion 11", "Level 11", "Item 11 1", "Item 11 2", "Item 11 3",
           "Trait 1", "Tier 1","Trait 2", "Tier 2","Trait 3", "Tier 3","Trait 4", "Tier 4","Trait 5", "Tier 5","Trait 6", "Tier 6",
           "Trait 7", "Tier 7","Trait 8", "Tier 8", "Placement"]
useless_traits = ["TFT13_HighRoller", "TFT13_MissMageTrait", "TFT13_MachineHerald", "TFT13_BloodHunter", "TFT13_JunkerKing"] #only one unit can have this trait so its not useful data, just repetitive

In [ ]:
def get_match_data(boards, index, matchIds):
  for m in tqdm(matchIds):
    region_long = 'AMERICAS'
    if ('EUW1' in m):
      region_long = 'EUROPE'
    elif ('KR' in m):
      region_long = 'ASIA'
    time.sleep(.5) #pause for api limits
    match_data = get_match(m, region_long)
    if (type(match_data) != int and match_data['info']['tft_set_number'] == 13): #there was a revival set 4 with completely different champions, filter it out
      for participant in match_data['info']['participants']:
        new_row = {col: 0 for col in columns}
        new_row['Placement'] = participant['placement'] #get placement data
        traits = 1
        for trait_num in range(len(participant['traits'])): #get all the traits
          if(participant['traits'][trait_num]['name'] not in useless_traits and participant['traits'][trait_num]['tier_current'] != 0): #make sure its not a useless trait or not activated
            new_row[f'Trait {traits}'] = participant['traits'][trait_num]['name'] #get the name
            new_row[f'Tier {traits}'] = participant['traits'][trait_num]['tier_current'] #get the tier
            traits = traits + 1
        for unit_num in range(len(participant['units'])): #get unit data
          new_row[f'Champion {unit_num+1}'] = participant['units'][unit_num]['character_id'] #get the name of the unit
          new_row[f'Level {unit_num+1}'] = participant['units'][unit_num]['tier'] #get the level of the unit
          for item_num in range(len(participant['units'][unit_num]['itemNames'])): #get the items on the unit
            if(participant['units'][unit_num]['itemNames'][item_num] == "TFT_Item_ThiefsGloves"): #thiefs_gloves gives two extra random items, only want thiefsgloves
              break
            new_row[f'Item {unit_num+1} {item_num+1}'] = participant['units'][unit_num]['itemNames'][item_num]
        boards.loc[index] = new_row
        index = index + 1
    else:
      if(type(match_data) == int):
        print(f"Match not completed: {match_data}") #print errors but continue
  return boards, index

In [22]:
boards = pd.DataFrame(columns=columns)
index = 0
section_one = matchIds[0:40000]
section_two = matchIds[40000:]
boards, index = get_match_data(boards, index, section_two)
boards

 47%|████▋     | 13139/27700 [4:45:39<5:51:00,  1.45s/it]

Match not completed: 504


 71%|███████▏  | 19790/27700 [8:31:11<4:51:50,  2.21s/it]

Match not completed: 504


 90%|█████████ | 25060/27700 [12:11:41<1:41:00,  2.30s/it]

...

100%|█████████▉| 27699/27700 [12:43:55<00:00,  1.34it/s]

Match not completed: 400


100%|██████████| 27700/27700 [12:43:55<00:00,  1.65s/it]

Match not completed: 400


,Champion 1,Level 1,Item 1 1,Item 1 2,Item 1 3,Champion 2,Level 2,Item 2 1,Item 2 2,Item 2 3,Champion 3,Level 3,Item 3 1,Item 3 2,Item 3 3,Champion 4,Level 4,Item 4 1,Item 4 2,Item 4 3,Champion 5,Level 5,Item 5 1,Item 5 2,Item 5 3,Champion 6,Level 6,Item 6 1,Item 6 2,Item 6 3,Champion 7,Level 7,Item 7 1,Item 7 2,Item 7 3,Champion 8,Level 8,Item 8 1,Item 8 2,Item 8 3,Champion 9,Level 9,Item 9 1,Item 9 2,Item 9 3,Champion 10,Level 10,Item 10 1,Item 10 2,Item 10 3,Champion 11,Level 11,Item 11 1,Item 11 2,Item 11 3,Trait 1,Tier 1,Trait 2,Tier 2,Trait 3,Tier 3,Trait 4,Tier 4,Trait 5,Tier 5,Trait 6,Tier 6,Trait 7,Tier 7,Trait 8,Tier 8,Placement
0,TFT13_Morgana,3,TFT_Item_Chalice,TFT_Item_StatikkShiv,0,TFT13_Singed,3,TFT_Item_IonicSpark,TFT_Item_SteraksGage,0,TFT13_Vex,2,0,0,0,TFT13_Rell,2,TFT_Item_GargoyleStoneplate,TFT_Item_NightHarvester,TFT_Item_GargoyleStoneplate,TFT13_RenataGlasc,3,TFT_Item_RapidFireCannon,TFT_Item_SpearOfShojin,TFT_Item_RabadonsDeathcap,TFT13_NunuWillump,2,0,0,0,TFT13_Heimerdinger,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,TFT13_Invoker,3,TFT13_Titan,1,0,0,0,0,0,0,0,0,0,0,0,0,7
1,TFT13_Rell,1,0,0,0,TFT13_Akali,2,0,0,0,TFT13_Beardy,2,TFT_Item_GargoyleStoneplate,TFT_Item_WarmogsArmor,0,TFT13_TwistedFate,3,TFT_Item_SpearOfShojin,TFT4_Item_OrnnZhonyasParadox,TFT_Item_Leviathan,TFT13_Ambessa,2,TFT_Item_SteraksGage,TFT_Item_LastWhisper,TFT_Item_StatikkShiv,TFT13_Illaoi,1,0,0,0,TFT13_Corki,1,TFT13_Item_ChallengerEmblemItem,0,0,TFT13_Rumble,1,TFT_Item_JeweledGauntlet,TFT_Item_UnstableConcoction,TFT_Item_SparringGloves,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,TFT13_Ambassador,1,TFT13_Challenger,3,TFT13_Scrap,1,TFT13_Squad,1,TFT13_Titan,2,TFT13_Warband,1,0,0,0,0,8
2,TFT13_Blue,2,TFT_Item_StatikkShiv,0,0,TFT13_Camille,2,0,0,0,TFT13_Gremlin,3,TFT_Item_Bloodthirster,TFT_Item_TitansResolve,TFT_Item_Artifact_ProwlersClaw,TFT13_FlyGuy,2,TFT13_Item_AmbusherEmblemItem,0,0,TFT13_Beardy,2,0,0,0,TFT13_Garen,2,TFT_Item_WarmogsArmor,TFT_Item_SpectralGauntlet,TFT_Item_GargoyleStoneplate,TFT13_Ekko,2,TFT_Item_Morellonomicon,TFT_Item_Bloodthirster,TFT_Item_Crownguard,TFT13_Illaoi,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,TFT13_Ambassador,1,TFT13_Ambusher,4,TFT13_Hoverboard,1,TFT13_Scrap,1,TFT13_Squad,1,TFT13_Titan,1,TFT13_Watcher,1,0,0,2
3,TFT13_Irelia,2,0,0,0,TFT13_Rell,2,0,0,0,TFT13_Akali,2,0,0,0,TFT13_Ezreal,2,0,0,0,TFT13_Ekko,2,TFT_Item_IonicSpark,TFT_Item_ChainVest,TFT_Item_Bloodthirster,TFT13_Illaoi,2,TFT7_Item_ShimmerscaleMogulsMail,TFT_Item_RedBuff,TFT_Item_BrambleVest,TFT13_Zoe,2,TFT_Item_RabadonsDeathcap,TFT_Item_SpearOfShojin,TFT_Item_GuinsoosRageblade,TFT13_Rumble,1,TFT13_Item_RebelEmblemItem,TFT_Item_Bloodthirster,TFT_Item_FrozenHeart,tft13_jinx,2,TFT_Item_Quicksilver,TFT_Item_LastWhisper,0,0,0,0,0,0,0,0,0,0,0,TFT13_Ambusher,1,TFT13_Rebel,3,TFT13_Scrap,1,TFT13_Titan,2,0,0,0,0,0,0,0,0,1
4,TFT13_Leona,2,0,0,0,TFT13_Rell,2,0,0,0,TFT13_RenataGlasc,2,TFT_Item_StatikkShiv,TFT_Item_RabadonsDeathcap,0,TFT13_NunuWillump,2,0,0,0,TFT13_Heimerdinger,2,TFT_Item_ArchangelsStaff,TFT_Item_BlueBuff,TFT_Item_HextechGunblade,tft13_elise,2,TFT_Item_ChainVest,TFT_Item_RedBuff,0,TFT13_Jayce,1,TFT9_Item_OrnnHorizonFocus,TFT13_Item_InvokerEmblemItem,TFT_Item_InfinityEdge,TFT13_Malzahar,2,TFT_Item_JeweledGauntlet,TFT_Item_RabadonsDeathcap,TFT_Item_Leviathan,TFT13_JayceSummon,1,0,0,0,0,0,0,0,0,0,0,0,0,0,TFT13_Academy,1,TFT13_Bruiser,1,TFT13_FormSwapper,1,TFT13_Invoker,3,TFT13_Titan,1,0,0,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183811,TFT13_Irelia,2,0,0,0,TFT13_Akali,2,0,0,0,TFT13_Sett,2,0,0,0,TFT13_Ekko,2,0,0,0,TFT13_Illaoi,2,TFT_Item_NightHarvester,TFT_Item_DragonsClaw,TFT_Item_DragonsClaw,TFT13_Zoe,2,TFT_Item_Leviathan,TFT_Item_MadredsBloodrazor,TFT_Item_JeweledGauntlet,TFT13_LeBlanc,2,TFT_Item_Stat

In [23]:
boards.to_csv('../data/dataset_NA_BR_EUW_KR_section2.csv', sep='\t', encoding='utf-8', index=False, header=True)